In [5]:
import numpy as np
from load_data import load
from counter import var_counter


In [6]:


#loading data from raw data files, no specificed path goes to V2 directory
[xcoord, ycoord, array4D]=load()

In [7]:
#stats accroos members
#computing mean and min-max ranges

[member_mean, member_max, member_min]=[np.mean(array4D,axis=0), np.amax(array4D,axis=0), np.amin(array4D,axis=0)]

In [8]:
var_counter(member_mean)

ValueError: operands could not be broadcast together with shapes (0,) (20,22,13) 